In [12]:
import numpy as np
from word2vec import * 
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import nltk
import gensim

# todo:1
class word_obj:
    cosine_sim=-100
    word=None
    
    def __init__(self, cosine_sim=-100, word='None'):
        self.cosine_sim = cosine_sim
        self.word = word
        
    def __str__(self):
        return self.word
    
    def __repr__(self): 
        return self.word

In [13]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
    return word_to_vec_map

In [14]:
def cosine_similarity(u, v):
    distance = 0.0
    # Compute the dot product between u and v
    dot = np.dot(u,v)
    # Compute the L2 norm of u
    norm_u = np.sqrt(np.sum(u * u))
    # Compute the L2 norm of v
    norm_v = np.sqrt(np.sum(v * v))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot / (norm_u * norm_v)
    return cosine_similarity

In [15]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    words = word_to_vec_map.keys()
    best_3_words_obj = [word_obj(), word_obj(), word_obj()] # Initialize best_words with None
    
    # convert words to lower case
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    # Get the word embeddings v_a, v_b, v_c and estimated v_d
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    e_d_ = e_c + e_b - e_a
    
    # loop over the whole word vector set
    for w in words:        
        # to avoid best_word being one of the input words, pass on them.
        if w in [word_a, word_b, word_c] :
            continue
        
        # Compute cosine similarity between the vector (e_b - e_a) and the w's vector - e_c)
        cosine_sim = cosine_similarity(e_d_, word_to_vec_map[w])
        
        # If the cosine_sim is more than the minimum in best_3, add it to best_3
        if cosine_sim > best_3_words_obj[2].cosine_sim:
            best_3_words_obj.append(word_obj(cosine_sim, w))
            best_3_words_obj = sorted(best_3_words_obj, key=lambda x: x.cosine_sim, reverse=True)[:3]
        
    return best_3_words_obj

In [16]:
def analogy_accuracy_rate(filepath):
    file = open(filepath, "r")
    lines = file.readlines()
    total = 0
    best_1 = 0
    best_3 = 0
    for line in lines:
        words = word_tokenize(line)
        if len(words) == 4:
            total += 1
            best_3_words_obj = complete_analogy(words[0], words[1], words[2], word_to_vec_map)
            best_3_words = [o.word for o in best_3_words_obj]
            print('given true a,b,c,d -> ', words)
            print('best_3 match for d item -> ', best_3_words)
            if best_3_words[0] == words[3]:
                best_1 += 1
            if words[3] in best_3_words:
                best_3 += 1
    print("best_1 accuracy: ", best_1/total)
    print("best_3 accuracy: ", best_3/total)       

In [17]:
word_to_vec_map = read_glove_vecs('/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/glove.6B.50d.txt')

In [23]:
len(word_to_vec_map)

400000

In [7]:
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/list1.txt")
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/list2.txt")
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/list3.txt")
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/list4.txt")

given true a,b,c,d ->  ['boy', 'girl', 'brother', 'sister']
best_3 match for d item ->  ['cousin', 'daughter', 'father']
given true a,b,c,d ->  ['brothers', 'sisters', 'his', 'her']
best_3 match for d item ->  ['her', 'she', 'herself']
given true a,b,c,d ->  ['dad', 'mom', 'grandfather', 'grandmother']
best_3 match for d item ->  ['grandmother', 'mother', 'granddaughter']
given true a,b,c,d ->  ['father', 'mother', 'groom', 'bride']
best_3 match for d item ->  ['bride', 'newlyweds', 'kissing']
given true a,b,c,d ->  ['grandpa', 'grandma', 'uncle', 'aunt']
best_3 match for d item ->  ['grandmother', 'grandfather', 'mother']
given true a,b,c,d ->  ['grandson', 'granddaughter', 'nephew', 'niece']
best_3 match for d item ->  ['niece', 'daughter', 'wife']
given true a,b,c,d ->  ['he', 'she', 'king', 'queen']
best_3 match for d item ->  ['queen', 'princess', 'daughter']
given true a,b,c,d ->  ['his', 'her', 'grandpa', 'grandma']
best_3 match for d item ->  ['grandma', 'mommy', 'mom']
given t

given true a,b,c,d ->  ['certain', 'uncertain', 'happy', 'sad']
best_3 match for d item ->  ['optimistic', 'happier', 'thrilled']
given true a,b,c,d ->  ['clear', 'unclear', 'big', 'small']
best_3 match for d item ->  ['biggest', 'bigger', 'ever']
given true a,b,c,d ->  ['comfortable', 'uncomfortable', 'tall', 'short']
best_3 match for d item ->  ['hairy', 'resembling', 'resembles']
given true a,b,c,d ->  ['competitive', 'uncompetitive', 'hot', 'cold']
best_3 match for d item ->  ['doggers', 'rodders', 'rodding']
given true a,b,c,d ->  ['consistent', 'inconsistent', 'funny', 'boring']
best_3 match for d item ->  ['silly', 'hilarious', 'amusing']
given true a,b,c,d ->  ['convincing', 'unconvincing', 'light', 'dark']
best_3 match for d item ->  ['fog', 'metallic', 'colour']
given true a,b,c,d ->  ['convenient', 'inconvenient', 'healthy', 'sick']
best_3 match for d item ->  ['cancer', 'worse', 'shy']
given true a,b,c,d ->  ['ethical', 'unethical', 'awake', 'asleep']
best_3 match for d ite

# Designed task 

In [8]:
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/adj_ comparative.txt")

given true a,b,c,d ->  ['angry', 'angrier', 'cheap', 'cheaper']
best_3 match for d item ->  ['costlier', 'pricier', 'affordably']
given true a,b,c,d ->  ['clever', 'cleverer', 'coarse', 'coarser']
best_3 match for d item ->  ['coarser', 'undyed', 'lucani']
given true a,b,c,d ->  ['costly', 'costlier', 'cute', 'cuter']
best_3 match for d item ->  ['adorable', 'perky', 'cuter']
given true a,b,c,d ->  ['dense', 'denser', 'dumb', 'dumber']
best_3 match for d item ->  ['dumber', 'conceited', 'egotistical']
given true a,b,c,d ->  ['fierce', 'fiercer', 'handy', 'handier']
best_3 match for d item ->  ['affordably', 'embraceable', 'smarttoaster']
given true a,b,c,d ->  ['happy', 'happier', 'hardy', 'hardier']
best_3 match for d item ->  ['amies', 'fownes', 'farre']
given true a,b,c,d ->  ['harsh', 'harsher', 'healthy', 'healthier']
best_3 match for d item ->  ['healthier', 'quicker', 'balanced']
given true a,b,c,d ->  ['hot', 'hotter', 'huge', 'huger']
best_3 match for d item ->  ['dwarfed', 'e

In [17]:
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/Text5.txt")

given true a,b,c,d ->  ['calm', 'calmly', 'complete', 'completely']
best_3 match for d item ->  ['nailed', 'inserted', 'completed']
given true a,b,c,d ->  ['apparent', 'apparently', 'slow', 'slowly']
best_3 match for d item ->  ['fast', 'quickly', 'getting']
given true a,b,c,d ->  ['amazing', 'amazingly', 'free', 'freely']
best_3 match for d item ->  ['allowing', 'restricted', 'allow']
given true a,b,c,d ->  ['cheerful', 'cheerfully', 'occasional', 'occasionally']
best_3 match for d item ->  ['frequent', 'jokes', 'endless']
given true a,b,c,d ->  ['most', 'mostly', 'fortunate', 'fortunately']
best_3 match for d item ->  ['terrified', 'unlucky', 'frightened']
given true a,b,c,d ->  ['obvious', 'obviously', 'serious', 'seriously']
best_3 match for d item ->  ['concerned', 'seriously', 'worse']
given true a,b,c,d ->  ['possible', 'possibly', 'quiet', 'quietly']
best_3 match for d item ->  ['sleepy', 'deserted', 'tranquil']
given true a,b,c,d ->  ['professional', 'professionally', 'immedia

In [19]:
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/Text6.txt")

given true a,b,c,d ->  ['acceptable', 'unacceptable', 'competitive', 'uncompetitive']
best_3 match for d item ->  ['competition', 'unfair', 'aggressive']
given true a,b,c,d ->  ['aware', 'unaware', 'certain', 'uncertain']
best_3 match for d item ->  ['any', 'presumably', 'specific']
given true a,b,c,d ->  ['reasonable', 'unreasonable', 'productive', 'unproductive']
best_3 match for d item ->  ['inefficient', 'exploitative', 'exploitive']
given true a,b,c,d ->  ['certain', 'uncertain', 'consistent', 'inconsistent']
best_3 match for d item ->  ['optimistic', 'pessimistic', 'remarkably']
given true a,b,c,d ->  ['clear', 'unclear', 'logical', 'illogical']
best_3 match for d item ->  ['logically', 'simplest', 'causal']
given true a,b,c,d ->  ['competitive', 'uncompetitive', 'informative', 'uninformative']
best_3 match for d item ->  ['well-done', 'unobjectionable', 'well-researched']
given true a,b,c,d ->  ['possibly', 'impossibly', 'efficient', 'inefficient']
best_3 match for d item ->  ['

In [16]:
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/I10 [verb_3pSg - Ved].txt")

given true a,b,c,d ->  ['adds', 'added', 'agrees', 'agreed']
best_3 match for d item ->  ['agreed', 'decided', 'intends']
given true a,b,c,d ->  ['allows', 'allowed', 'announces', 'announced']
best_3 match for d item ->  ['announcing', 'announced', 'last']
given true a,b,c,d ->  ['appears', 'appeared', 'applies', 'applied']
best_3 match for d item ->  ['rules', 'laws', 'apply']
given true a,b,c,d ->  ['appoints', 'appointed', 'asks', 'asked']
best_3 match for d item ->  ['he', 'father', 'she']
given true a,b,c,d ->  ['becomes', 'became', 'believes', 'believed']
best_3 match for d item ->  ['recently', 'joined', 'had']
given true a,b,c,d ->  ['considers', 'considered', 'consists', 'consisted']
best_3 match for d item ->  ['consisting', 'consist', 'consisted']
given true a,b,c,d ->  ['contains', 'contained', 'continues', 'continued']
best_3 match for d item ->  ['continuing', 'continue', 'continued']
given true a,b,c,d ->  ['creates', 'created', 'decides', 'decided']
best_3 match for d i

In [11]:
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/name_nationality.txt")

given true a,b,c,d ->  ['aristotle', 'greek', 'balzac', 'french']
best_3 match for d item ->  ['spanish', 'french', 'portuguese']
given true a,b,c,d ->  ['beethoven', 'german', 'caesar', 'roman']
best_3 match for d item ->  ['mercenaries', 'forces', 'foreign']
given true a,b,c,d ->  ['confucius', 'chinese', 'copernicus', 'polish']
best_3 match for d item ->  ['russian', 'foreign', 'currency']
given true a,b,c,d ->  ['darwin', 'english', 'depp', 'american']
best_3 match for d item ->  ['plays', 'actor', 'comedy']
given true a,b,c,d ->  ['descartes', 'french', 'dickens', 'english']
best_3 match for d item ->  ['british', 'american', 'britain']
given true a,b,c,d ->  ['dostoyevsky', 'russian', 'edison', 'american']
best_3 match for d item ->  ['union', 'motors', 'ukrainian']
given true a,b,c,d ->  ['einstein', 'jewish', 'euclid', 'greek']
best_3 match for d item ->  ['netzarim', 'synagogues', 'settlements']
given true a,b,c,d ->  ['fermi', 'italian', 'galilei', 'italian']
best_3 match for

In [10]:
analogy_accuracy_rate("/Users/nehakardam/Documents/UWclasses /EE517 NLP/Lab 1/Things_color.txt")

given true a,b,c,d ->  ['ant', 'black', 'apple', 'red']
best_3 match for d item ->  ['orange', 'brand', 'white']
given true a,b,c,d ->  ['blackboard', 'black', 'blood', 'red']
best_3 match for d item ->  ['causes', 'and', 'face']
given true a,b,c,d ->  ['blueberry', 'blue', 'broccoli', 'green']
best_3 match for d item ->  ['red', 'trimmed', 'brown']
given true a,b,c,d ->  ['bruise', 'blue', 'cabbage', 'green']
best_3 match for d item ->  ['orange', 'apples', 'olive']
given true a,b,c,d ->  ['carrot', 'orange', 'cauliflower', 'green']
best_3 match for d item ->  ['purple', 'florets', 'pink']
given true a,b,c,d ->  ['celery', 'green', 'cherry', 'red']
best_3 match for d item ->  ['white', 'gray', 'jackson']
given true a,b,c,d ->  ['chocolate', 'brown', 'cloud', 'white']
best_3 match for d item ->  ['guard', 'foley', 'reserve']
given true a,b,c,d ->  ['coal', 'black', 'coffee', 'black']
best_3 match for d item ->  ['shirt', 'shirts', 'dress']
given true a,b,c,d ->  ['cranberry', 'red', 'c